# $\mathbf{\{0,1\}}$ - Integer Programming with Complete Constraint Matrices

## Random Informed Guesses

We begin by enumerating (given an integer $m$) all possible linear combinations on the columns of the complete matrix. Based on those, we try to filter the ones that are withing the ranges:
* $\mathbf{1} \cdot 2^{m-2} \leq \mathbf{b} \leq \mathbf{1} \cdot  2^{m-1}$
* $\mathbf{1} \cdot 2^{m-3} \leq \mathbf{b} \leq \mathbf{1} \cdot  2^{m-2}$
* $\mathbf{1} \cdot 2^{m-4} \leq \mathbf{b} \leq \mathbf{3} \cdot  2^{m-2}$
* $\vdots$

In [1]:
from helpers import *
from tqdm import tqdm
import numpy as np
import pickle
import json
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
A = make_complete(5, dt=int)
for row in A:
    for el in row:
        print(el, end=" & ")
    print("\\\\")

0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & \\
0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & \\
0 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & 0 & 0 & 1 & 1 & \\
0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & 0 & 1 & \\


In [7]:
feasRHS_4 = []
for i in range(1,5):
    with open('ComputationalResults/feasRHS_4/feasRHS_4_res'+str(i)+'_4', 'rb') as f:
        res = pickle.load(f)
        # if i == 1:
        #     feasRHS_5 = res
        for rhs in res:
            feasRHS_4.append(json.loads(rhs))

feasRHS_4 = np.array(feasRHS_4).T


In [8]:
m = 4
CONDITIONS_ADD = {}
CONDITIONS_ONE = {}
feasRHS = {}
for i in range(1, m+1):
    for j in range(1, m+1):
        if i < j:
            CONDITIONS_ADD[f'2^(m-{j}) <= b <= 2^(m-{i})'] = (2**(m-j),2**(m-i))
    CONDITIONS_ONE[f"0 <= b <= 2^(m-{i})"] = 2**(m-i)

for k, v in CONDITIONS_ADD.items():
    feasRHS[k] = ((np.array(((feasRHS_4 >= v[0]) * (feasRHS_4 <= v[1])), dtype=int).sum(0))==m).sum()

for k, v in CONDITIONS_ONE.items():
    feasRHS[k] = ((np.array((feasRHS_4 <= v), dtype=int).sum(0))==m).sum()

pick_in = open('ComputationalResults/ranges4/ranges4.pkl', 'rb')
ranges4 = pickle.load(pick_in)

In [9]:
Comp_dict = {}
num_combos = 2**(2**m - 1)
columns = ['# of lin combos', '# of RHS', 'lin. combos/RHS', '% of feasible RHS', '% of lin. combos']
ranges = ranges4
for k, v in CONDITIONS_ADD.items():
    num_rhs = (v[1] - v[0] + 1)**m
    Comp_dict[k] = (ranges[k], feasRHS[k], ranges[k]/feasRHS[k], feasRHS[k]/num_rhs, ranges[k]/num_combos)

for k, v in CONDITIONS_ONE.items():
    num_rhs = (v + 1)**m
    Comp_dict[k] = (ranges[k], feasRHS[k], ranges[k]/feasRHS[k], feasRHS[k]/num_rhs, ranges[k]/num_combos)

comps = pd.DataFrame.from_dict(Comp_dict, orient='index', columns=columns)

# for k in CONDITIONS_ADD.keys():
#     Comp_dict[k] = (ranges5[k], result_dict[k], result_dict[k]/ranges5[k])

In [29]:
import math
for row in comps5.values:
    for val in row[:-1]:
        print("${:.3e}$".format(val), end=' & ')
    print("${:.3e}$".format(row[-1]), end=' ')
    print('\\\\', end="\n")


$5.464e+08$ & $3.475e+04$ & $1.572e+04$ & $5.886e-01$ & $2.545e-01$ \\
$2.055e+09$ & $1.094e+05$ & $1.877e+04$ & $2.948e-01$ & $9.568e-01$ \\
$2.145e+09$ & $1.463e+05$ & $1.466e+04$ & $1.927e-01$ & $9.987e-01$ \\
$2.147e+09$ & $1.593e+05$ & $1.348e+04$ & $1.519e-01$ & $9.999e-01$ \\
$4.635e+08$ & $3.125e+03$ & $1.483e+05$ & $1.000e+00$ & $2.159e-01$ \\
$5.438e+08$ & $1.653e+04$ & $3.289e+04$ & $9.836e-01$ & $2.532e-01$ \\
$5.463e+08$ & $2.723e+04$ & $2.006e+04$ & $8.309e-01$ & $2.544e-01$ \\
$1.762e+06$ & $2.430e+02$ & $7.253e+03$ & $1.000e+00$ & $8.207e-04$ \\
$2.179e+06$ & $1.024e+03$ & $2.128e+03$ & $1.000e+00$ & $1.015e-03$ \\
$6.995e+03$ & $3.200e+01$ & $2.186e+02$ & $1.000e+00$ & $3.257e-06$ \\
$2.147e+09$ & $1.668e+05$ & $1.287e+04$ & $1.175e-01$ & $1.000e+00$ \\
$5.464e+08$ & $3.475e+04$ & $1.572e+04$ & $5.886e-01$ & $2.545e-01$ \\
$2.233e+06$ & $2.780e+03$ & $8.032e+02$ & $8.896e-01$ & $1.040e-03$ \\
$9.736e+03$ & $2.380e+02$ & $4.091e+01$ & $9.794e-01$ & $4.534e-06$ \\
$2.030

In [36]:
for row in comps5.axes[0]:
    row = row.replace("<=", "\\leq")
    row = row.replace("(", "{")
    row = row.replace(")", "}")
    print("$"+row+"$ &", end=" \n")

$2^{m-2} \leq b \leq 2^{m-1}$ & 
$2^{m-3} \leq b \leq 2^{m-1}$ & 
$2^{m-4} \leq b \leq 2^{m-1}$ & 
$2^{m-5} \leq b \leq 2^{m-1}$ & 
$2^{m-3} \leq b \leq 2^{m-2}$ & 
$2^{m-4} \leq b \leq 2^{m-2}$ & 
$2^{m-5} \leq b \leq 2^{m-2}$ & 
$2^{m-4} \leq b \leq 2^{m-3}$ & 
$2^{m-5} \leq b \leq 2^{m-3}$ & 
$2^{m-5} \leq b \leq 2^{m-4}$ & 
$0 \leq b \leq 2^{m-1}$ & 
$0 \leq b \leq 2^{m-2}$ & 
$0 \leq b \leq 2^{m-3}$ & 
$0 \leq b \leq 2^{m-4}$ & 
$0 \leq b \leq 2^{m-5}$ & 


| Range of Values               | Number of combinations | Number of RHS |
|-------------------------------|------------------------|---------------|
| $2^{m-2} \leq b \leq 2^{m-1}$ | $546433766$            | $34754$       |
| $2^{m-3} \leq b \leq 2^{m-2}$ | $463546849$            | $3125$        |
| $2^{m-4} \leq b \leq 2^{m-3}$ | $1762382$              | $146320$      |
| $2^{m-5} \leq b \leq 2^{m-4}$ | $6995$                 | $243$         |

| Range of Values               | Number of combinations | Number of RHS |
|-------------------------------|------------------------|---------------|
| $2^{m-3} \leq b \leq 2^{m-1}$ | $2054665066$           | $109443$      |
| $2^{m-4} \leq b \leq 2^{m-2}$ | $543768948$            | $16532$       |
| $2^{m-5} \leq b \leq 2^{m-3}$ | $2178649$              | $1024$        |

| Range of Values               | Number of combinations | Number of RHS |
|-------------------------------|------------------------|---------------|
| $2^{m-4} \leq b \leq 2^{m-1}$ | $2144799235$           | $146320$      |
| $2^{m-5} \leq b \leq 2^{m-2}$ | $546271135$            | $27228$       |

| Range of Values               | Number of combinations | Number of RHS |
|-------------------------------|------------------------|---------------|
| $2^{m-5} \leq b \leq 2^{m-1}$ | $2147321017$           | $159291$      |

| Range of Values         | Number of combinations | Number of RHS |
|-------------------------|------------------------|---------------|
| $0 \leq b \leq 2^{m-1}$ | $2147483648$           | $166817$      |
| $0 \leq b \leq 2^{m-2}$ | $546433766$            | $34754$       |
| $0 \leq b \leq 2^{m-3}$ | $2232875$              | $2780$        |
| $0 \leq b \leq 2^{m-4}$ | $9736$                 | $238$         |
| $0 \leq b \leq 2^{m-5}$ | $203$                  | $32$          |

In [29]:
import json
with open('ComputationalResults/feasRHS_4', 'rb') as f:
    feasRHS_4 = pickle.load(f)
    f.close()
feas = []
for rhs in feasRHS_4:
    rhs = json.loads(rhs)
    feas.append(rhs)
feas = np.array(feas)

KeyboardInterrupt: 

Fractionality depends on the determinant of the matrix because of Kramer's rule

## Claims & Intuitions

$\text{\bf Claim}$: If a right hand side $\mathbf{b} = (b_1, \dots, b_m)$ is feasible then $\max_i\{b_i\}-\min_i\{b_i\} \leq 2^{m-2}$

$\text{\bf Proof}$: Without loss of generality, assume that $\mathbf{b}$ is sorted. We claim, that if $\mathbf{b}$ is a feasible right hand side, then $b_1-b_m \leq 2^{m-2}$.

Suppose for the sake of contradiction that this is not the case. Then $b_1-b_m > 2^{m-2}$, and in particular we have that $b_1 > 2^{m-2}$.

Let $\mathbf{x}$ be a solution for $\mathbf{Ax} = \mathbf{b}$, and $C$ be the set of columns used to obtain that solution. Let $C_1 \subseteq C$ be the subset of columns with a leading one. It must hold that $|C_1|>2^{m-2}$. In addition, we can define $\mathbf{A}_1$ to be the submatrix of $\mathbf{A}$ composed of columns containing a $1$ as the first element.

We have that each row of $\mathbf{A}_1$ contains exactly $2^{m-2}$ ones, except the first one which contains $2^{m-1}$ ones. Additionally the elements of $C_1$ are columns of $\mathbf{A}_1$. Then since $|C_1|>2^{m-2}$, then each row of the sub-matrix of $\mathbf{A}_1$ formed by the columns in $C_1$ contains at least $b_1-2^{m-2}$ ones.

In particular this implies that $b_m \geq b_1-2^{m-2}$ and it follows that $b_1 - b_m \leq 2^{m-2}$. qed

$\textbf{Intuition 1}$: Let $\mathbf{b} \in \mathbb{N}^m$ be a vector and $\mathbf{b}_{\text{sort}$ be the sorted version of $\mathbf{b}$. Then $\mathbf{b}$ is feasible $\iff \mathbf{b}_{\text{sort}$ is feasible.